In [ ]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

import re
from tqdm import tqdm_notebook
import os

In [ ]:
# DTM 문서 단어 행렬
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer    
tfvector = TfidfTransformer(smooth_idf=False)

In [ ]:
vectorizer=TfidfVectorizer(ngram_range=(1, 3),
    min_df=2,       
    max_features=10000,
    sublinear_tf=True,
    lowercase=False,
    use_idf=True)

# Directory 변경


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
cd gdrive/My\ Drive/kb_data

/content/gdrive/My Drive/kb_data


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [ ]:
prepared_train = pd.read_csv('prepared_train.csv', encoding='utf-8')

In [ ]:
del prepared_train['Unnamed: 0']

In [ ]:
%%time
vec_train = vectorizer.fit_transform(prepared_train['orgin'])

CPU times: user 49.6 s, sys: 1.5 s, total: 51.1 s
Wall time: 51.1 s


In [ ]:
#X_train = prepared_train.drop(['year_month', 'text', 'morph', 'orgin', 'smishing'], axis='columns')
X_origin = vec_train
Y_label = prepared_train['smishing']

In [ ]:
# train - test 분류
# 인덱싱 문제로 순서대로 train / test split
%%time
X_train, X_test, Y_train, Y_test = train_test_split(X_origin,
                                                    Y_label,
                                                    test_size = 0.2,
                                                    shuffle = False)
                                                    # random_state = 1825)

CPU times: user 50.5 ms, sys: 966 µs, total: 51.4 ms
Wall time: 51 ms


# Checking False records for using SVM

In [ ]:
submissions_list= os.listdir('svm_models/submissions')

In [ ]:
len(submissions_list)

30

In [ ]:
# 모델마다 false record 탐색
neg_dict = dict()
for sub in submissions_list:
    try:
        df = pd.read_csv('svm_models/submissions/'+sub, engine='python')
        df.set_index('id', inplace= True)
        # 각 모델마다 false record를 담은 임시 리스트
        temp_list = list()
        for idx in df.index:
            if (df['pred'][idx] != Y_test[idx]):
                # 없을 때만 추가
                temp_list.append(idx)
        neg_dict[str(sub)] = temp_list
    except:
        pass
    


In [ ]:
# 30개 모델이 공통적으로 틀린 record 파악 share_neg = list()
temp_list = list()
share_neg = list()

for csv_list in neg_dict:
    for r in neg_dict[csv_list]:
        temp_list.append(r)

temp_list.sort()

# 틀린 record 중복 제거: total_list
total_list = set(temp_list)
# id_num = dict()
# key: record ID, value: 틀린 갯수 (0 ~ 30)
id_num = dict()
for i in total_list:
    num = temp_list.count(i)
    id_num[i] = num


In [ ]:
#over_list: 30개 모델 모두 틀린 record
over_list = list()
for i in id_num:
    if (id_num[i] == 30):
        over_list.append(i)
        

In [ ]:
len(over_list)

80

In [ ]:
print(over_list)

[256005, 287239, 275982, 257551, 292368, 283677, 271390, 280609, 272932, 255546, 255040, 275522, 284737, 255064, 259673, 282717, 275557, 275060, 287888, 258711, 293527, 285862, 257704, 259761, 260286, 283839, 255168, 255179, 280781, 258259, 259805, 281311, 283363, 267492, 266982, 260332, 260335, 250099, 256760, 286984, 288523, 255764, 257813, 258851, 270633, 259884, 256819, 262471, 256338, 258386, 259932, 290652, 271715, 257893, 259432, 289651, 258933, 255353, 259985, 255894, 260502, 253852, 258462, 274849, 253858, 257966, 267695, 261553, 243634, 257463, 260036, 261578, 258012, 256990, 242668, 279533, 257519, 270326, 288757, 255484]


In [ ]:
false_pos_text = list()
false_neg_text = list()
false_pos_idx = list()
false_neg_idx = list()

for idx in over_list:
    # false negative: 실제로는 양성인데, 검사결과 음성
    if (prepared_train['smishing'][idx] == 1):
        false_neg_idx.append(idx)
        false_neg_text.append(prepared_train['orgin'][idx])
    # false positve: 실제로는 음성인데, 검사결과 양성
    elif (prepared_train['smishing'][idx] == 0):
        false_pos_idx.append(idx)
        false_pos_text.append(prepared_train['orgin'][idx])

In [ ]:
len(false_pos_idx)

1

In [ ]:
len(false_neg_idx)

79

In [ ]:
print(false_pos_text)

['안녕하십니까XXX영등포하이테크지점 입니다청년자영업자를 위한 저금리 대출이 출시 되었습니다1)대상:만39세이하7년이내기업2)기간:3년 일시상환3)금리:1년0.5%  2년부터1.5%4)한도:최대 3억본 대출은 제조신성장유망창업기업등이며 신용보증기금과 기술신용보증기금보증서 대출과연계하여 진행하며 한도 2천억 범위내로 운영하고 있습니다상담전화는 XX-XXX-XXX4533문의 주시면 최선을 다해 상담드리겠습니다']


In [ ]:
print(false_neg_text)

['(광고) 한국경제TV 와우스탁론Fed  기준금리 동결 2일 미국 8월 연방공개시장위원회(FOMC)에 대해 시장이 예상한 수준에서 크게 벗어나지 않았다고 분석미국 XXX(Fed)은 이틀간 FOMC 정례회의를 열고 기준금리를 현 1.752.00%로 유지하기로 만장일치로 결정했다고 밝혔다.뉴욕 증시다우산업 25333.82 81.37 -0.32%나스닥종합 7707.29 35.50 0.46%SP500 2813.36 2.93 -0.1%5월 프로모션 상품 RMS사용료  연장수수료  중도상환수수료  업계최저금리 월 0.4% 사용가능 한종목 100% 매수가능 최장 5년까지 이용가능 증권사 신용미수스탁론   매도없이 저금리 대환가능와우스탁론 박숙향XX-XXX-XXX XXX-XXX-XXX 수신거부포함제목없음', '안녕하세요 좀전에 XXX린 XXX입니다 추가 자금필요하시거나 기존고금리상품 대환가능하셔요연락주시면 친절히 상담드리겠습니다 오늘도 좋은하루 되세요안녕하세요 좀전에 XXX', '(광고)안녕하세요대성금융 XXX팀장입니다.상담:XXX-XXX-XXX카톡ID :good8424반갑습니다최근 장기적인 경기침체와 내수경기 불황으로 인하여 많은 분들께서 금융사를 통하여 자금을 이용하고 계십니다.저희 대성금융은 작업대출 전문업체로서 더이상 금융권에 대출이 불가능하신 분들이나 연체자 무직자 주부등 금융권쪽에서대출이 불가 하신분들에 한하여 필요한 자금 300만5000만원을 100% 맞추어 드리고 있습니다.저희는 선입금이나 절대 금전적인 요구 없습니다대출에에 필요한 서류는 본인이 직접 은행 방문 하실때 구비출해서 담당자 만나뵙고 전달하면 됩니다.4대보험 미가입시 저희가 은행방문전 필요서류 보내드립니다.저희는 개인신용도로 대출을 받게 도와 드리는게 아니라 각은행마다 개인신용평가시스템(CSS)에 의해 대출이 가능한 상품으로개인평점을 단기간에 상향시켜 대출가능하게 진행해 드립니다.기존채무 저희 업채에서 상환처리하여 대출진행이 원활히 가능합니다단 선입금 서류 필요없이 대출금 받으신후 저희에게 작업비

# Checking False records for using LGBM

In [ ]:
submissions_list= os.listdir('lgbm_models/submissions')[:30]

In [ ]:
len(submissions_list)

30

In [ ]:
# 모델마다 false record 탐색
neg_dict = dict()
for sub in submissions_list:
    try:
        df = pd.read_csv('lgbm_models/submissions/'+sub, engine='python')
        df.set_index('id', inplace= True)
        # 각 모델마다 false record를 담은 임시 리스트
        temp_list = list()
        for idx in df.index:
            if (df['pred'][idx] != Y_test[idx]):
                # 없을 때만 추가
                temp_list.append(idx)
        neg_dict[str(sub)] = temp_list
    except:
        pass
    


In [ ]:
# 30개 모델이 공통적으로 틀린 record 파악 share_neg = list()
temp_list = list()
share_neg = list()

for csv_list in neg_dict:
    for r in neg_dict[csv_list]:
        temp_list.append(r)

temp_list.sort()

# 틀린 record 중복 제거: total_list
total_list = set(temp_list)
# id_num = dict()
# key: record ID, value: 틀린 갯수 (0 ~ 30)
id_num = dict()
for i in total_list:
    num = temp_list.count(i)
    id_num[i] = num


In [ ]:
#over_list: 30개 모델 틀린 record
over_list2 = list()
for i in id_num:
    if (id_num[i] == 30):
        over_list2.append(i)

In [ ]:
false_pos_text2 = list()
false_neg_text2 = list()
false_pos_idx2 = list()
false_neg_idx2 = list()

for idx in over_list2:
    # false negative: 실제로는 양성인데, 검사결과 음성
    if (prepared_train['smishing'][idx] == 1):
        false_neg_idx2.append(idx)
        false_neg_text2.append(prepared_train['orgin'][idx])
    # false positve: 실제로는 음성인데, 검사결과 양성
    elif (prepared_train['smishing'][idx] == 0):
        false_pos_idx2.append(idx)
        false_pos_text2.append(prepared_train['orgin'][idx])

In [ ]:
len(false_pos_idx2)

6

In [ ]:
len(false_neg_idx2)

196

# 공통적으로 틀린 record

In [ ]:
sum = list()
for i in over_list:
    for j in over_list2:
        if i == j:
            sum.append(i)

In [ ]:
print(sum)

[256005, 287239, 257551, 292368, 275522, 255064, 259673, 282717, 275060, 287888, 258711, 293527, 257704, 259761, 260286, 283839, 255179, 280781, 258259, 259805, 250099, 255764, 258851, 270633, 257893, 259432, 289651, 258933, 253852, 258462, 257966, 243634, 257463, 260036, 258012, 256990, 242668, 255484]


In [ ]:
len(sum)

38

In [ ]:
sum_fal_pos = list()
sum_fal_neg = list()
sum_fal_pos_idx = list()
sum_fal_neg_idx = list()

for idx in sum:
    # false negative: 실제로는 양성인데, 검사결과 음성
    if (prepared_train['smishing'][idx] == 1):
        sum_fal_neg_idx.append(idx)
        sum_fal_neg.append(prepared_train['orgin'][idx])

    # false positve: 실제로는 음성인데, 검사결과 양성
    elif (prepared_train['smishing'][idx] == 0):
        sum_fal_pos_idx.append(idx)
        sum_fal_pos.append(prepared_train['orgin'][idx])

In [ ]:
sum_fal_neg_idx

In [ ]:
sum_fal_neg

In [ ]:
sum_fal_pos_idx

[]

# LIME

## LIME for SVM


In [ ]:
models = os.listdir('svm_models/[7578, 3023, 5514]')
models_list = [x for x in models if x.endswith(".pkl")]

In [ ]:
svm_model = models_list[0]

In [ ]:
svm_model = joblib.load('svm_models/[7578, 3023, 5514]/'+svm_model)

In [ ]:
class_names = ['normal', 'smishing']
explainer = LimeTextExplainer(class_names=class_names)
classifier = make_pipeline(vectorizer, svm_model)

### False_Negative record

In [ ]:
for idx in false_neg_idx:
    print(idx, '스미싱 여부: ', prepared_train['smishing'][idx])
    explanation = explainer.explain_instance(prepared_train['orgin'][idx], classifier.predict_proba, num_features=6)
    print(explanation.show_in_notebook(text=True))
    explanation.as_list()
    print()
    print()

### False_Positive record

In [ ]:
for idx in false_pos_idx:
    print('스미싱 여부: ', prepared_train['smishing'][idx])
    explanation = explainer.explain_instance(prepared_train['orgin'][idx], classifier.predict_proba, num_features=6)
    print(explanation.show_in_notebook(text=True))
    explanation.as_list()
    print()
    print()

## LIME For LGBM

In [ ]:
models = os.listdir('lgbm_models/[3696, 2744, 6389]')
models_list = [x for x in models if x.endswith(".pkl")]

In [ ]:
lgbm_model = models_list[0]

In [ ]:
lgbm_model = joblib.load('lgbm_models/[3696, 2744, 6389]/'+lgbm_model)

In [ ]:
class_names = ['normal', 'smishing']
explainer = LimeTextExplainer(class_names=class_names)
classifier = make_pipeline(vectorizer, lgbm_model)

### False_Negative record

In [ ]:
for idx in false_neg_idx2:
    print(idx, '스미싱 여부: ', prepared_train['smishing'][idx])
    explanation = explainer.explain_instance(prepared_train['orgin'][idx], classifier.predict_proba, num_features=6)
    print(explanation.show_in_notebook(text=True))
    explanation.as_list()
    print()
    print()

### False_Positive record

# 쓰지 않을 코딩

In [ ]:
# 간단하게 합집합으로 false record 탐색

# 참값: Y_test, 예측값: df['pred']
neg_list = list()
for sub in submissions_list:
    df = pd.read_csv('svm_models/submissions/'+sub)
    df.set_index('id', inplace= True)
    for idx in df.index:
        if (df['pred'][idx] != Y_test[idx]):
            # 없을 때만 추가
            if idx not in neg_list:
                neg_list.append(idx)

In [ ]:
neg_list.sort()

In [ ]:
len(neg_list)

222

In [ ]:
false_pos_text = list()
false_neg_text = list()
false_pos_idx = list()
false_neg_idx = list()

for idx in neg_list:

    if (prepared_train['smishing'][idx] == 1):
        if prepared_train['orgin'][idx] not in false_pos_text:
            false_pos_text.append(prepared_train['orgin'][idx])
            false_pos_idx.append(idx)
    elif (prepared_train['smishing'][idx] == 0):
        if prepared_train['orgin'][idx] not in false_neg_text:
            false_neg_text.append(prepared_train['orgin'][idx])
            false_neg_idx.append(idx)

In [ ]:
print(len(false_pos_text))
print(len(false_pos_idx))
print(len(false_neg_text))
print(len(false_neg_idx))